In [ ]:
#Installing needed libraries
!pip install geopandas

In [ ]:
#Importing needed libraries
import pandas as pd
import geopandas as gpd

In [ ]:
#Loading CEQR projects data 
ceqr_projects = pd.read_csv('CEQR_Projects.csv')
ceqr_projects.head()

,CEQR,Project Name,Project Description,Borough,Lead Agency,URL
0,04DCP052Q,Brookville Rezoning,To amend the Zoning Map to rezone 88 blocks fr...,Queens,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...
1,93TLC001Y,Community Cab Proposal,Proposal for a new classification of For-Hire ...,Citywide,Taxi and Limousine Commission,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...
2,99BSA079K,874 Bedford Avenue,Proposed Residential Dwellings in an M1-1 Zone,Brooklyn,Board of Standards and Appeals,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...
3,16DCP043R,22 Norman Place,"The Applicant, 22 Norman Place LLC, is request...",Staten Island,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...
4,94DBS006M,Downtown Lower Manhattan BID,Development of a Business Improvement District,Manhattan,Department of Small Business Services,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...


In [ ]:
#Loading Projects that are only rezoning projects
ceqr_projects = ceqr_projects[ceqr_projects['Project Name'].str.contains("Rezoning")]

In [ ]:
#Loading all project locations
ceqr_locations = pd.read_csv('CEQR_Project_Locations.csv')
ceqr_locations.head()

,CEQR,Project Name,Block,Lot,Community District,House Number,Street Name,Postcode
0,82-270M,46-50 Ganesvoort Street Project,643.0,43.0,MN03,NaN,NaN,NaN
1,82-270M,46-50 Ganesvoort Street Project,643.0,43.0,MN02,NaN,NaN,NaN
2,82-270M,46-50 Ganesvoort Street Project,643.0,49.0,MN03,NaN,NaN,NaN
3,82-270M,46-50 Ganesvoort Street Project,643.0,49.0,MN02,NaN,NaN,NaN
4,82-270M,46-50 Ganesvoort Street Project,643.0,54.0,MN03,NaN,46-50 GANESVOORT STREET,NaN


In [ ]:
#Grouping all addresses as per CEQR number and retaining only first block and lot information 
ceqr_locations=ceqr_locations.groupby('CEQR').first()
ceqr_locations.head()

,Project Name,Block,Lot,Community District,House Number,Street Name,Postcode
CEQR,,,,,,,
00BOE001M,Agency Facility,1077.0,43.0,MN04,NaN,NaN,NaN
00BSA001K,Restaurant on 8th Avenue,1112.0,52.0,BK07,NaN,"1638 8TH AVENUE, WEST 100-0 NORT",NaN
00BSA002M,286 Spring Street,579.0,5.0,MN02,286,SPRING STREET,10013.0
00BSA004M,625 Greenwich Street,602.0,56.0,MN02,625,GREENWICH STREET,10014.0
00BSA005M,East Houston Street Variance,412.0,53.0,MN03,215,EAST HOUSTON STREET,10002.0


In [ ]:
#Loading project milestones datasets
ceqr_milestones = pd.read_csv('CEQR_Project_Milestones.csv')
ceqr_milestones.head()

,CEQR,Project Name,Milestone Name,Milestone Date
0,09DCP036M,246 Eleventh Avenue,Negative Declaration,05/04/2009
1,94BSA034Q,Lucille Roberts Fitness Center,EAS,NaN
2,94BSA037Q,Lucille Roberts Fitness Center,Revised EAS,07/06/1994
3,09DCP048M,Fountain House,Lead Agency Letter,03/31/2009
4,13BSA002K,910 Manhattan Avenue,Negative Declaration,01/08/2013


In [ ]:
#Converting to datetime
ceqr_milestones['Milestone Date']= pd.to_datetime(ceqr_milestones['Milestone Date'])

In [ ]:
#Filtering to take only the last entered stage for the project
ceqr_milestones=ceqr_milestones.groupby('CEQR').last()

In [ ]:
#Joining all CEQR related datasets together
ceqr_data = ceqr_projects.merge(ceqr_locations,how='left',on='CEQR')
ceqr_data = ceqr_data.merge(ceqr_milestones,how='left',on='CEQR')
ceqr_data = ceqr_data.drop(columns=['Project Name_y','Project Name_x'])
ceqr_data.head()

,CEQR,Project Description,Borough,Lead Agency,URL,Block,Lot,Community District,House Number,Street Name,Postcode,Project Name,Milestone Name,Milestone Date
0,04DCP052Q,To amend the Zoning Map to rezone 88 blocks fr...,Queens,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,NaN,NaN,QN13,NaN,"PORTIONS OF 84 BLOCKS, GENERALLY",NaN,Brookville Rezoning,EAS,2004-05-05
1,06DCP020X,Zoning map amendment from R4 to R4A for a 3 bl...,Bronx,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,5559.0,157.0,BX10,NaN,NaN,NaN,Brush Avenue Rezoning,Negative Declaration,2005-09-12
2,06DCP091Q,To rezone all portions of 21 blocks in the wes...,Queens,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,NaN,NaN,NaN,NaN,NaN,NaN,Middle Village Follow Up Rezoning,EAS,2006-03-31
3,09DCP077Q,An amendment to the Zoning Map to extend a C1-...,Queens,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,12775.0,17.0,QN13,NaN,NaN,NaN,Springfield Boulevard Rezoning,Lead Agency Letter,2009-06-11
4,01DCP076X,A zoning map change that would remove an aband...,Bronx,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,3943.0,199.0,BX09,NaN,PURDY STREET,NaN,1701 Purdy Street Rezoning,EAS,2005-01-25


In [ ]:
#Loading BBL shapefiles
bbl = gpd.read_file('MapPLUTO.shp')
bbl.head()

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthCent,HealthArea,Sanitboro,SanitDistr,SanitSub,Address,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,SPDist3,LtdHeight,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,...,ProxCode,IrrLotCode,LotType,BsmtCode,AssessLand,AssessTot,ExemptTot,YearBuilt,YearAlter1,YearAlter2,HistDist,Landmark,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLA,PFIRM15_FL,Version,DCPEdited,Latitude,Longitude,Notes,Shape_Leng,Shape_Area,geometry
0,MN,1097,11,104,129,1001,02,3,10019,E054,18,15,4500,1,04,3B,683 11 AVENUE,M2-4,None,None,None,None,None,CL,None,None,None,N,U2,07,0,X,"CONSOLIDATED EDISON CO.OF N.Y.,INC.",112893,0,0,0,0,0,0,...,0,N,5,0,0.0,0.0,0.0,0,0,0,None,None,0.0,0.00,5.0,0.0,1,1.010970e+09,0,0129,985389,218351,8c,None,105N050,10405,None,0.0,None,1,None,None,20v3,None,40.765999,-73.995888,None,1516.586853,112892.966455,"POLYGON ((985681.146 218305.545, 985582.473 21..."
1,BK,5527,50,311,246,2000,20,47,11204,E247,62,31,8110,3,11,1D,1965 62 STREET,R5,None,None,None,None,None,None,None,None,None,N,A5,01,0,None,"XIE, JOSEPH JIANDONG",1800,1440,0,1440,0,0,0,...,3,N,5,2,17880.0,57960.0,0.0,1925,1999,0,None,None,0.8,1.25,0.0,2.0,3,3.055270e+09,0,0246,988564,164706,22d,None,310A028,31703,None,0.0,None,1,None,None,20v3,None,40.618755,-73.984461,None,240.542515,1886.597524,"POLYGON ((988603.467 164740.225, 988540.364 16..."
2,BX,3251,421,207,409,2000,10,11,10468,L046,52,21,310,2,07,2A,WEST 205 STREET,R7-1,None,None,None,None,None,None,None,None,None,N,U7,07,0,C,NYC TRANSIT AUTHORITY,20385,0,0,0,0,0,0,...,0,Y,5,0,0.0,0.0,0.0,0,0,0,None,None,0.0,3.44,0.0,4.8,2,2.032510e+09,0,0409,1015075,258766,1d,None,213 043,21201,None,0.0,None,1,None,None,20v3,t,40.876873,-73.888535,None,626.788403,20384.789265,"POLYGON ((1015172.497 258742.899, 1015039.514 ..."
3,QN,15100,175,484,1072.02,1006,27,31,11693,E266,100,45,3612,None,None,None,159 STREET,R3-2,PARK,None,None,None,None,None,None,None,None,Y,U7,07,0,X,NYC TRANSIT AUTHORITY,142309,0,0,0,0,0,0,...,0,N,5,0,0.0,0.0,0.0,0,0,0,None,None,0.0,0.50,0.0,1.0,4,4.151000e+09,0,107202,1033369,170879,24c,Y,421 999,45806,None,0.0,None,1,1,1,20v3,t,40.635564,-73.823026,None,2091.812869,142215.984020,"POLYGON ((1033442.342 170926.651, 1033442.520 ..."
4,QN,12882,20,413,616.01,3000,29,31,11422,L165,105,44,3531,4,13,5C,129 AVENUE,R2A,None,None,None,None,None,None,None,None,None,N,U2,07,0,X,"CONSOLIDATED EDISON CO.OF N.Y.,INC.",23681,0,0,0,0,0,0,...,0,N,5,0,0.0,0.0,0.0,0,0,0,None,None,0.0,0.50,0.0,1.0,4,4.128820e+09,0,061601,1059100,187089,19c,None,414 063,45405,None,0.0,None,1,None,None,20v3,None,40.679876,-73.730138,None,813.274279,23681.100545,"POLYGON ((1059021.836 187078.195, 1059025.156 ..."


In [ ]:
#Merging BBL and CEQR data with Block and Lot Information
ceqr_bbl = pd.merge(bbl, ceqr_data, on=['Block','Lot'], how='inner', suffixes=['_and', '_or'])
ceqr_bbl.head()

,Borough_and,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthCent,HealthArea,Sanitboro,SanitDistr,SanitSub,Address,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,SPDist3,LtdHeight,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,...,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLA,PFIRM15_FL,Version,DCPEdited,Latitude,Longitude,Notes,Shape_Leng,Shape_Area,geometry,CEQR,Project Description,Borough_or,Lead Agency,URL,Community District,House Number,Street Name,Postcode,Project Name,Milestone Name,Milestone Date
0,MN,202,18,103,29,3000,02,1,10013,E009,5,15,7700,1,03,1A,54 BOWERY,C6-1G,None,None,None,None,None,None,None,None,None,N,K7,05,0,None,HSBC BANK USA,8804,17616,17616,0,0,17616,0,...,2.00,3.44,6.0,4.8,1,1.002020e+09,0,0029,985200,200186,12c,None,101S032,10105,None,0.000000e+00,None,1,None,None,20v3,t,40.716141,-73.996573,None,409.843942,9277.572064,"POLYGON ((985238.078 200123.707, 985146.820 20...",19DCP009K,"The applicant, Slwest Old Fulton, LLC., reques...",Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK02,NaN,NaN,NaN,50 Old Fulton Rezoning,EAS,2019-10-25
1,MN,2050,100,109,227,2000,06,7,10031,L023,30,17,720,1,09,3B,13 HAMILTON TERRACE,R6A,None,None,None,None,None,None,None,None,None,N,D1,03,0,None,"HAMILTON HEIGHTS ASSOCIATES, LLC",9639,49616,0,49616,0,0,0,...,5.15,3.00,0.0,3.0,1,1.020500e+09,0,0227,999027,238782,6a,None,111S039,10709,None,0.000000e+00,None,1,None,None,20v3,None,40.822065,-73.946610,None,511.963108,12292.509388,"POLYGON ((999058.637 238821.849, 999082.008 23...",17DCP176K,"The applicant, MADDD Equities, LLC, is seeking...",Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK02,NaN,NaN,NaN,Tillary and Prince Streets Rezoning,EAS,2017-09-15
2,MN,567,5,102,59,1000,02,2,10003,E033,6,15,5700,1,02,3A,29 5 AVENUE,R10,None,None,None,None,None,None,None,None,None,N,D9,03,0,None,HDH3.2 LLC,2633,16247,1000,15247,1000,0,0,...,6.17,10.00,0.0,10.0,1,1.005670e+09,0,0059,985538,206356,12c,None,103 022,10207,None,0.000000e+00,None,1,None,None,20v3,t,40.733076,-73.995353,None,258.927336,2720.867294,"POLYGON ((985590.296 206343.231, 985577.312 20...",10DCP019Q,To rezone approx 890 acres (on 238 blocks) of ...,Queens,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,QN01,NaN,NaN,NaN,Astoria Rezoning,EAS,2010-01-22
3,BX,2624,41,201,75,1000,07,17,10455,E073,40,23,4000,2,01,2B,600 EAST 156 STREET,R8A,None,None,None,None,None,None,None,None,None,N,D9,03,0,None,600 EAST 156TH STREET HOUSING DEV. FUND CORP.,23000,209853,22750,0,0,0,15300,...,9.12,6.02,0.0,6.5,2,2.026240e+09,0,0075,1009341,237548,6c,None,209N096,21003,None,2.026240e+09,01/29/2016,1,None,None,20v3,None,40.818654,-73.909349,None,679.378721,24490.931164,"POLYGON ((1009438.294 237464.320, 1009340.860 ...",17DCP025X,"The applicant, 600 Associates, LLC, is seeking...",Bronx,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BX01,NaN,NaN,NaN,600 East 156th Street Rezoning,EAS,2016-11-10
4,BX,3237,38,207,269,2000,10,14,10463,E081,52,21,320,2,07,1A,2546 BAILEY AVENUE,R6,None,None,None,None,None,None,None,None,None,N,A5,01,0,None,2546 ENTERPRISES LLC,1150,2160,0,1440,0,0,0,...,1.88,2.43,0.0,4.8,2,2.032370e+09,0,0269,1009743,255479,3c,None,214 035,21111,None,0.000000e+00,None,1,None,None,20v3,None,40.867869,-73.907829,None,145.904798,1095.820525,"POLYGON ((1009773.289 255479.506, 1009761.083 ...",05DCP040K,An amendment to the Zoning Map to change an M1...,Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK04,70,WYCKOFF AVENUE,11237.0,Wyckoff Avenue Rezoning,Lead Agency Letter,2005-01-13


In [ ]:
#Installing necessary geospatial libraries
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
!pip install pyspark
!pip install fiona
!pip install shapely

In [ ]:
#Importing needed libraries
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

import seaborn as sns
sns.set(style="whitegrid")

import fiona
import fiona.crs
import shapely
import rtree

import pandas as pd
import geopandas as gpd

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#Loading census tract information
nyc_counties = ["005","059","061","081","085","047","119"]
spatial_data = gpd.read_file('tl_2019_36_bg.shp')
spatial_data = spatial_data.loc[spatial_data['COUNTYFP'].isin(nyc_counties)]
spatial_data.to_file("block_group.json", driver="GeoJSON")

In [ ]:
#ceqr_bbl.to_csv('ceqr_bbl.csv')
#ceqr_bbl.to_file("ceqr_bbl.json", driver="GeoJSON")

In [ ]:
#Converting to GeoJSON 

#bbls = gpd.read_file('ceqr_bbl.json').to_crs(fiona.crs.from_epsg(2263))
block_groups = gpd.read_file('block_group.json').to_crs(fiona.crs.from_epsg(2263))

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
#Creating rtree indexes for spatial matching of BBLs and Census Tracts
index = rtree.Rtree()
for idx,geometry in enumerate(block_groups.geometry):
    index.insert(idx, geometry.bounds)

In [ ]:
index.bounds

[912190.8893736254, 113288.79017251907, 1144239.485961955, 437426.18835837935]

In [ ]:
idx

8339

In [ ]:
#Spatial matching of BBLs and Census Tracts

import csv
import pyproj
import shapely.geometry as geom


proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    

counts = []

for i, row in ceqr_bbl.iterrows():
  p = geom.Point(proj(float(row['Longitude']), float(row['Latitude'])))
  match = None
  for idx in index.intersection((p.x, p.y, p.x, p.y)):
    if block_groups.geometry[idx].contains(p):
      match = block_groups.GEOID[idx]
      counts.append(match)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


In [ ]:
#Creating new column with Census Tract GEOIDs
ceqr_bbl['GEOID'] = counts
ceqr_bbl.head()

,Borough_and,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthCent,HealthArea,Sanitboro,SanitDistr,SanitSub,Address,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,SPDist3,LtdHeight,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,...,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLA,PFIRM15_FL,Version,DCPEdited,Latitude,Longitude,Notes,Shape_Leng,Shape_Area,geometry,CEQR,Project Description,Borough_or,Lead Agency,URL,Community District,House Number,Street Name,Postcode,Project Name,Milestone Name,Milestone Date,GEOID
0,MN,202,18,103,29,3000,02,1,10013,E009,5,15,7700,1,03,1A,54 BOWERY,C6-1G,None,None,None,None,None,None,None,None,None,N,K7,05,0,None,HSBC BANK USA,8804,17616,17616,0,0,17616,0,...,3.44,6.0,4.8,1,1.002020e+09,0,0029,985200,200186,12c,None,101S032,10105,None,0.000000e+00,None,1,None,None,20v3,t,40.716141,-73.996573,None,409.843942,9277.572064,"POLYGON ((985238.078 200123.707, 985146.820 20...",19DCP009K,"The applicant, Slwest Old Fulton, LLC., reques...",Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK02,NaN,NaN,NaN,50 Old Fulton Rezoning,EAS,2019-10-25,360610029003
1,MN,2050,100,109,227,2000,06,7,10031,L023,30,17,720,1,09,3B,13 HAMILTON TERRACE,R6A,None,None,None,None,None,None,None,None,None,N,D1,03,0,None,"HAMILTON HEIGHTS ASSOCIATES, LLC",9639,49616,0,49616,0,0,0,...,3.00,0.0,3.0,1,1.020500e+09,0,0227,999027,238782,6a,None,111S039,10709,None,0.000000e+00,None,1,None,None,20v3,None,40.822065,-73.946610,None,511.963108,12292.509388,"POLYGON ((999058.637 238821.849, 999082.008 23...",17DCP176K,"The applicant, MADDD Equities, LLC, is seeking...",Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK02,NaN,NaN,NaN,Tillary and Prince Streets Rezoning,EAS,2017-09-15,360610227002
2,MN,567,5,102,59,1000,02,2,10003,E033,6,15,5700,1,02,3A,29 5 AVENUE,R10,None,None,None,None,None,None,None,None,None,N,D9,03,0,None,HDH3.2 LLC,2633,16247,1000,15247,1000,0,0,...,10.00,0.0,10.0,1,1.005670e+09,0,0059,985538,206356,12c,None,103 022,10207,None,0.000000e+00,None,1,None,None,20v3,t,40.733076,-73.995353,None,258.927336,2720.867294,"POLYGON ((985590.296 206343.231, 985577.312 20...",10DCP019Q,To rezone approx 890 acres (on 238 blocks) of ...,Queens,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,QN01,NaN,NaN,NaN,Astoria Rezoning,EAS,2010-01-22,360610059001
3,BX,2624,41,201,75,1000,07,17,10455,E073,40,23,4000,2,01,2B,600 EAST 156 STREET,R8A,None,None,None,None,None,None,None,None,None,N,D9,03,0,None,600 EAST 156TH STREET HOUSING DEV. FUND CORP.,23000,209853,22750,0,0,0,15300,...,6.02,0.0,6.5,2,2.026240e+09,0,0075,1009341,237548,6c,None,209N096,21003,None,2.026240e+09,01/29/2016,1,None,None,20v3,None,40.818654,-73.909349,None,679.378721,24490.931164,"POLYGON ((1009438.294 237464.320, 1009340.860 ...",17DCP025X,"The applicant, 600 Associates, LLC, is seeking...",Bronx,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BX01,NaN,NaN,NaN,600 East 156th Street Rezoning,EAS,2016-11-10,360050075001
4,BX,3237,38,207,269,2000,10,14,10463,E081,52,21,320,2,07,1A,2546 BAILEY AVENUE,R6,None,None,None,None,None,None,None,None,None,N,A5,01,0,None,2546 ENTERPRISES LLC,1150,2160,0,1440,0,0,0,...,2.43,0.0,4.8,2,2.032370e+09,0,0269,1009743,255479,3c,None,214 035,21111,None,0.000000e+00,None,1,None,None,20v3,None,40.867869,-73.907829,None,145.904798,1095.820525,"POLYGON ((1009773.289 255479.506, 1009761.083 ...",05DCP040K,An amendment to the Zoning Map to change an M1...,Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK04,70,WYCKOFF AVENUE,11237.0,Wyckoff Avenue Rezoning,Lead Agency Letter,2005-01-13,360050269002


In [ ]:
#Saving file
ceqr_bbl.to_csv('ceqr_bbl.csv')